In [5]:
#################Reading the dataset and considering the items that have more than one click for a unique query #################
#################################################################################################################################
from collections import defaultdict
import csv
import pandas as pd
import json
import numpy as np
import random
count = 0
dataset = defaultdict(list)
print ("start...")
with open('st_oms_signals_2020_apr_sep.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
#         print(row)
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            #if the number of clicks is more than 2 for a specific item
            if row[-1] >= '2':
                count += 1
                dataset[row[0]].append(row[1])
                line_count += 1

              

print(count)
# print(set(dataset.keys()))
print('done...')

start...
Column names are 
59205591
done...


In [6]:
################# product detail dataset and create "taxonomy trajectory", "product_category(leaf nodes)", and item2prodcategory #################
#################################################################################################################################

print('reading data....')
product_details = pd.read_csv('catalog_data_20210125.csv')
print('done...')
dict_omsid_to_disc = {}
dict_prodcategory_to_trajectory = {}
dict_item_to_pd = {}
product_categories = {}
is_in = set()
items_all = set()


counter = 0
for i, row in enumerate(product_details.iterrows()):
#     if i > 100:
#         break

    item_id = str(row[1][0])    
    taxonomy = str(row[1][4]).lower()
    collection = str(row[1][5]).lower()
    title = str(row[1][1]).lower()
    brand = str(row[1][2]).lower()
    color = str(row[1][-1]).lower()

    
    dict_omsid_to_disc[item_id] = [title,' '.join(str(taxonomy).split('>')), ' '.join(str(collection).split('/')), brand+ " </mid> "+ color]
#     print(dict_omsid_to_disc[item_id])


    if '>' in str(taxonomy):
        prod_cat = taxonomy.split('>')[-1]
        dict_item_to_pd[str(item_id)] = prod_cat

        if prod_cat in is_in:
            continue
        else:
            is_in.add(prod_cat)
            product_categories[prod_cat] = counter
            counter += 1
            dict_prodcategory_to_trajectory[prod_cat] = taxonomy.split('>')

            
    
print(len(dict_omsid_to_disc.keys()))    

with open('dict_prodcategory_to_trajectory.json', 'w') as out_file:
    json.dump(dict_prodcategory_to_trajectory, out_file)
out_file.close()
            
            
with open('product_categories.json', 'w') as out_file:
    json.dump(product_categories, out_file)
out_file.close()

with open('dict_item_to_productcategory.json', 'w') as out_file:
    json.dump(dict_item_to_pd, out_file)
out_file.close()

print("done....")


reading data....
done...
4781036
done....


In [7]:
count = 0
dataset_impr = {}
with open('st_oms_signals_impr_2020_apr_sep.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
#         if line_count > 1000:
#             break
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            count += 1
#             print (row)
            
            vals = row[1].split("|")
            new_val = []
            for i, v in enumerate(vals):
                if len(v) > 0 and i < 10:
                    new_val.append(v)
                    
            if len(new_val) >= 3:
#                 print(new_val)
                dataset_impr[row[0]] = new_val
                line_count += 1

# print(dataset_impr)             
print (line_count)
print('done...')

Column names are 
1413429
done...


In [8]:
################################## Creating the index and value files for the labels ##################################
#################################################################################################################################
print("dict_omsid_to_disc", len(dict_omsid_to_disc.keys()))

import re
import numpy as np
from collections import Counter
def create_value_index(labels):
    # mapping the last label into the (value , index) pairs
    # it has only one so session gets[0]
    lst_labels = labels

    values_ = np.zeros(10)
    indexes_ = np.zeros(10, dtype=int)
    rnd_labels = set()

    for i in range(10):
        try:
            if lst_labels[i] not in rnd_labels:
                rnd_labels.add(lst_labels[i])
                values_[i] = 1
                indexes_[i] = lst_labels[i]

            else:
                for ii in range(10):
                    ind_rnd = random.randint(0, len(product_categories) - 1)
                    if ind_rnd not in rnd_labels:
                        rnd_labels.add(ind_rnd)
                        break
                indexes_[i] = ind_rnd
                values_[i] = 1e-7

        except:
            for ii in range(10):
                ind_rnd = random.randint(0, len(product_categories) - 1)
                if ind_rnd not in rnd_labels:
                    rnd_labels.add(ind_rnd)
                    break
            indexes_[i] = ind_rnd
            values_[i] = 1e-7


    new_indexes_ii = sorted(range(len(indexes_)), key=lambda k: indexes_[k])
    new_values_ = [values_[i] for i in new_indexes_ii]
    new_indexes_ = [indexes_[i] for i in new_indexes_ii]
    
    indexes = defaultdict(list)
    values = defaultdict(list)
    indexes['ind'].extend(new_indexes_)
    values['val'].extend(new_values_)
    final_labels = {'indexes': new_indexes_, 'values': new_values_}
    return final_labels


################################################################
################################################################
################################################################
print("start....")
new_dataset = []
max_val = []

queries1 = set(dataset_impr.keys())
queries2 = set(dataset.keys())

print (len(queries2-queries1))

for i,query in enumerate(dataset):
#     if i > 1000:
#         break
    new_sample = {}
    new_sample['query'] = query

    categories = Counter()
    for item in dataset[query]:
        try:
            categories[product_categories[dict_item_to_pd[item]]] += 1
        except:
            pass
    
    
    try:
        imprs = dataset_impr[query]
    except:
        imprs = []
    
#     print (imprs)
    if len(imprs) > 0:
        short_disc = []
        for im in imprs:
            if len(im) > 0:
                try:
                    omsid_to_disc = dict_omsid_to_disc[str(im)]
                    short_disc.append(omsid_to_disc)
                except:
                    pass
#                 print(omsid_to_disc)

        if len(categories) > 0:
            prod_cat = dict((k, v) for k, v in categories.items() if v >=0).keys()
            final_labels = create_value_index(list(prod_cat))
            new_sample['label'] = final_labels
            new_sample['short_disc'] = short_disc
            new_dataset.append(new_sample)


    
print (len(new_dataset))
print("done....")

dict_omsid_to_disc 4781036
start....
738523
1398259
done....


In [19]:
################################## Shuffle and extracting 0.95 percent for training ##################################
######################################################################################################################

from numpyencoder import NumpyEncoder

random.shuffle(new_dataset)
print('start...')
file_ = open("train_dataset_10.txt", 'w')
count = 0
for i, sample in enumerate(new_dataset):
    if i >= 0.5 * len(new_dataset):
        break
#     print (sample)
    file_.write(json.dumps(sample, cls= NumpyEncoder))
    file_.write('\n')
    count += 1
file_.close()

print (count)
print("done....")

start...
699130
done....


In [20]:
################################## Shuffle and extracting 0.05 percent for validation ##################################
########################################################################################################################

from numpyencoder import NumpyEncoder

print('start...')
file_ = open("validation_set_10.txt", 'w')
count = 0
for i, sample in enumerate(new_dataset):
    if i > len(new_dataset) * 0.8:
        sample["Bucket"] = ['1']
        file_.write(json.dumps(sample, cls= NumpyEncoder))
        file_.write('\n')
        count += 1
file_.close()

print (count)
print("done....")

start...
279651
done....


In [21]:
print('start...')
file_ = open("validation_set_10.txt", 'r').read().split("\n")
# print (file_[0:100])
count = 0
data_query = []
for i, sample in enumerate(file_):
    if len(sample) > 0: 
        line = json.loads(sample)
        data_query.append([line['query']])


import csv  
    
# field names  
fields = ['query']  
    
# name of csv file  
filename = "test_queries.csv"
    
# writing to csv file  
with open(filename, 'w') as csvfile:  
    # creating a csv writer object  
    csvwriter = csv.writer(csvfile)  
        
    # writing the fields  
    csvwriter.writerow(fields)  
        
    # writing the data rows  
    csvwriter.writerows(data_query) 
    
print("done....")

start...
done....


In [22]:
import csv
from numpyencoder import NumpyEncoder


from csv import reader
# open file in read mode
buckets = set(['Head', 'Mid', 'Tail'])

volume = {}
with open('head_mid_tail.csv', 'r') as read_obj:
    csv_reader = csv.reader(read_obj)
    for i, row in enumerate(csv_reader):
        if i > 0:
            if row[8] in buckets:
                volume[row[1]] = row[8]
            

# print(len())

print('start...')
file_ = open("validation_set_bucket_10.txt", 'w')
count = 0
for i, sample in enumerate(new_dataset):
    try:
        if i > len(new_dataset) * 0.5:

            sample["Bucket"] = [volume[sample['query']]]
            file_.write(json.dumps(sample, cls= NumpyEncoder))
            file_.write('\n')
            count += 1
    except:
        pass
file_.close()

print (count)
print("done....")

start...
139321
done....


In [24]:
import csv
from numpyencoder import NumpyEncoder


from csv import reader
# open file in read mode

buckets_attrs = {"num_attr_f" :"numerical",  "num_dim_f":"numerical", "upc_f" : "ID", "sku_f":"ID", "modelNumber_f":"ID"
                 ,"brand_f":"brand", "product":"product", "color-material":"color"}


print(buckets_attrs.keys())
volume = {}
with open('df_seg.csv', 'r') as read_obj:
    csv_reader = csv.reader(read_obj)
    for i, row in enumerate(csv_reader):
        if i > 0:
            volume[row[2]] = row[1]
            
#         if i == 10:
#             break
# print(volume)

print('start...')
file_ = open("validation_set_bucket_segments.txt", 'w')
count = 0
for i, sample in enumerate(new_dataset):
    try:
        if i > len(new_dataset) * 0.5:

            attrs = volume[sample['query']].split(';')
            attrs_bucket = set()
            for att in attrs:
#                 print (att)
                if att in buckets_attrs.keys():
                    attrs_bucket.add(buckets_attrs[att])
            
            if len(list(attrs_bucket)) > 0:
                sample["Bucket"] = list(attrs_bucket)
                file_.write(json.dumps(sample, cls= NumpyEncoder))
                file_.write('\n')
                count += 1
            
    except:
        pass
file_.close()

print (count)
print("done....")

dict_keys(['num_attr_f', 'num_dim_f', 'upc_f', 'sku_f', 'modelNumber_f', 'brand_f', 'product', 'color-material'])
start...
9276
done....
